# 1.下载数据

In [6]:
import csv,xlrd,xlwt,datetime,time,re
import numpy as np

import pandas as pd


#一、导入指导价


#二、打开对方清单
def suanqingdan():
    suanqingdan=0
    global url
    shijian0=time.strftime('%Y-%m-%d',time.localtime(time.time()))
    shijian=datetime.datetime.strptime(shijian0 , "%Y-%m-%d")#转为日期格式
    shijian=str(shijian.strftime("%Y-%m-%d"))
   # shijian='2019-04-302'
    
    
    #jin=pd.read_table('jin.csv',sep=',')
    jin=pd.read_excel('jin.xls',encoding='GB18030')
    jin=jin[jin['客户名称']!='作废']
    jin=jin.set_index('客户名称')



#2.2确认列名，到时候可以删除
    url=input('文件名:')
    if url=='':
        print('dd')
        
        
    else:    
        qing=pd.read_excel(url)
        qing=qing[['客户名称','生效起始日','生效截至日','可用额度']]  #,
        for i in range(0,len(qing)):
            if qing.loc[i,'客户名称'] == '作废':
                qing.drop(i)

        
        qing=qing.set_index('客户名称')
        qing.rename(columns={'可用额度':shijian}, inplace=True)
        del jin['生效起始日']
        del jin['生效截至日']
        print(jin.head())
        
        print(qing.head())
        print(jin.tail())
        
        print(qing.tail())

        jin=jin.join(qing,how='outer')


    jin.to_excel('jin.xls')
suanqingdan()


文件名:
dd


# 分析变动

In [27]:
import numpy,math
jin=pd.read_excel('jin.xls',encoding='GB18030')
shijian0=time.strftime('%Y-%m-%d',time.localtime(time.time()))
shijian=datetime.datetime.strptime(shijian0 , "%Y-%m-%d")#转为日期格式
shijian=str(shijian.strftime("%Y-%m-%d"))
jincolumn=jin.columns.values.tolist()  
jincolumn.remove('客户名称')
jincolumn.remove('生效起始日')
jincolumn.remove('生效截至日')
print('已经收集的日期有：')
print(jincolumn)
zuijinri=max(jincolumn)
jincolumn.remove(zuijinri)
shangyiri=max(jincolumn)
print('最近日期：')

print(zuijinri)
print('上一日期：')
print(shangyiri)


jin[shangyiri]=jin[shangyiri].astype('float')/100000000
jin[zuijinri]=jin[zuijinri].astype('float')/100000000
print('\n')
for i in range(0,len(jin)):
    if math.isnan(jin.loc[i,zuijinri]) and  not math.isnan(jin.loc[i,shangyiri]):
             print('删除名单')
             print('%s     %s     %s    %s'%(jin.loc[i,'客户名称'],jin.loc[i,shangyiri],jin.loc[i,'生效起始日'],jin.loc[i,'生效截至日']))
             jin.loc[i,'改变']=1
    elif not math.isnan(jin.loc[i,zuijinri]) and  math.isnan(jin.loc[i,shangyiri]):    
             print('新增名单:')
             print('%s     %s     %s    %s'%(jin.loc[i,'客户名称'],jin.loc[i,zuijinri],jin.loc[i,'生效起始日'],jin.loc[i,'生效截至日']))
             jin.loc[i,'改变']=1
    jin.loc[i,'%s变动'%zuijinri]=(jin.loc[i,zuijinri]-jin.loc[i,shangyiri])



    
    
jin=jin[jin['改变']!=1]
jin = jin.sort_values(by=('%s变动'%zuijinri), ascending=False)
print('\n')
print('余额总共新增：%s 亿元'%(jin['%s变动'%shijian].sum()))
jin=jin.round({'%s变动'%shijian:2})
jin=jin.round({zuijinri:2})
jin=jin.round({shangyiri:2})


jin=jin[['客户名称','生效截至日',zuijinri,'%s变动'%shijian]]
jin=jin.reset_index(drop = True)

print(jin.head(15))
print('\n')
print(jin.tail(15))

已经收集的日期有：
['2019-04-29', '2019-04-30']
最近日期：
2019-04-30
上一日期：
2019-04-29


新增名单:
阳泉煤业集团财务有限责任公司     1.59     20190428    20190831


余额总共新增：3.690060939100004 亿元
                客户名称     生效截至日  2019-04-30  2019-04-30变动
0         中信银行股份有限公司  20190930        9.51          2.55
1         浙商银行股份有限公司  20190930        6.36          2.07
2         中国银行股份有限公司  20190930        9.87          1.96
3         平安银行股份有限公司  20190930       26.76          1.91
4         南京银行股份有限公司  20190930        1.60          1.60
5         北京银行股份有限公司  20191130        2.90          1.42
6         泰安银行股份有限公司  20191031        1.31          1.20
7     南昌农村商业银行股份有限公司  20191130        1.16          0.95
8         洛阳银行股份有限公司  20191130        0.95          0.77
9         招商银行股份有限公司  20190930        1.49          0.73
10        上海银行股份有限公司  20191030        0.70          0.70
11          国投财务有限公司  20190930        7.95          0.62
12    厦门农村商业银行股份有限公司  20190930        2.89          0.58
13  江苏紫金农村商业银行股份有限公司  20190930        0.68